In [1]:
import pandas as pd
from ge

In [6]:
tw_df = pd.read_csv('Data/alltweets.csv',sep=',')
tw_df['content']

0       Weiter geht’s! Heute mit #Rapid #Innovation au...
1       If you think to hire someone how is an #expert...
2       Want to get more involved with @swopla but cou...
3       Knowledge has a beginning but no end. Life lon...
4       National Honor Society induction today at #RHS...
5       Congrats to our new officer board! We’re excit...
6       The greatest DevOps challenge - maturing proce...
7       Can I pick 23 &amp; 6 😂😂💨 Wat about you ? #can...
8       #Atlanta All your car buying needs🔴🔴#Atlanta #...
9       Look at the pride on her face! Jayla is workin...
10      This is what 24/7/365 #HVAC #service looks lik...
11      Meet our newest Law Enforcement deputies! They...
12      Look at this pair of “off white” @VolvoTrucks ...
13      @usps online tracking system down ... and hold...
14      Your #Welcome To Our #Warm #Home.... The Potte...
15      Wanted: Your stories of service! PenFed Digita...
16      #Service Our friends @amplify4good are looking...
17      Do you